In [239]:
# part 1 (0) :
#==========================================================
# 📌 모듈 불러오기
#==========================================================
import os, sys, random, json, subprocess
from typing import List, Dict, Optional, Tuple

import numpy as np

# 안전 임포트 (있으면 쓰고, 없으면 None)
try:
    import torch
except Exception:
    torch = None

# LangChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
print("✅ 모듈 불러오기 - 완료")


✅ 모듈 불러오기 - 완료


In [240]:
# part 1 (1):
#==========================================================
# 📌 시드 고정(재현성)
#==========================================================
SEED = int(os.getenv("SEED", "2024"))
random.seed(SEED)
np.random.seed(SEED)
if torch is not None:
    try:
        if torch.cuda.is_available():
            torch.manual_seed(SEED)
            torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass
print("✅ 시드 고정 - 완료")


✅ 시드 고정 - 완료


In [241]:
# part 1 (2) 
#===============================================================================
# ▶ 작업환경 플래그
#===============================================================================
IS_GOOGLE = True if 'google.colab'           in sys.modules   else False
IS_KAGGLE = True if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ    else False
IS_LOCAL  = True if not (IS_GOOGLE or IS_KAGGLE)              else False
print(f"✅ IS_GOOGLE={IS_GOOGLE}, IS_KAGGLE={IS_KAGGLE}, IS_LOCAL={IS_LOCAL}")


✅ IS_GOOGLE=False, IS_KAGGLE=False, IS_LOCAL=True


In [242]:
# part 1 (3) 
#==========================================================
# 📌 상수설정 (모델/토큰/메시지) + api_token.txt 로드
#==========================================================
API_TOKEN_PATH  = os.getenv("API_TOKEN", "./api_token.txt")
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY", "")

if not OPENAI_API_KEY and os.path.exists(API_TOKEN_PATH):
    try:
        with open(API_TOKEN_PATH, "r", encoding="utf-8") as f:
            _k = f.read().strip()
        if _k:
            os.environ["OPENAI_API_KEY"] = _k
            OPENAI_API_KEY = _k
            print("✅ OpenAI API 키를 api_token.txt에서 로드했습니다.")
    except Exception as e:
        print(f"⚠️ API 토큰 파일 읽기 실패: {e}")

if OPENAI_API_KEY:
    print("✅ OpenAI API 키 준비됨")
else:
    print("⚠️ OPENAI_API_KEY 미설정 — 실행 전 반드시 설정 또는 api_token.txt 준비")

# 🔧 모델명
CHAT_MODEL_NAME  = os.getenv("OPENAI_CHAT_MODEL", "gpt-4o-mini")
EMBED_MODEL_NAME = os.getenv("OPENAI_EMBED_MODEL", "text-embedding-3-small")

# 🔢 토큰/파라미터
MAX_OUTPUT_TOKENS = int(os.getenv("MAX_OUTPUT_TOKENS", "800"))
TEMPERATURE       = float(os.getenv("TEMPERATURE", "0.2"))

# 🛡 도메인 제한 메시지
OFF_TOPIC_MESSAGE = "치매관련 상담만 가능합니다."

# 🧭 고정 질문 상수 (요청 형식)
Q1 = '자주 쓰던 물건 이름이 갑자기 생각안 난적이 있나요?'
Q2 = '대화중단어가 잘 떠오르지 않아서 곤란했던 적이 있나요?'
Q3 = '가족이나 지인이 평소와 다르다고 한적이 있나요?'
Q4 = '최근에 불편했던 점이나 걱정되는 점이 있나요?'

GUIDE_QUESTIONS = [Q1, Q2, Q3, Q4]
print("✅ 상수설정 - 완료")


✅ OpenAI API 키 준비됨
✅ 상수설정 - 완료


In [243]:
# part 2 (1) 
#==========================================================
# 📌 페르소나, 템플릿, 정책 (섹션3: 사전고정 불릿 사용)
#==========================================================
from langchain_core.prompts import ChatPromptTemplate

SYSTEM_PERSONA = '''\
당신은 한국어로 상담하는 의학(치매진단) 상담 챗봇입니다.
- 역할: 환자/보호자의 서술(STT 텍스트)을 읽고, 지정된 템플릿으로 간결하고 체계적인 요약을 만듭니다.
- 태도: 정중하고 전문적이며, 과도한 확신/진단을 피하고 안전수칙을 강조합니다.
- 범위: 치매, 경도인지장애, 관련 증상/검사/일상 안전/가족 교육과 같은 주제에 한정합니다.
- 금지: 치매상담과 무관한 정보(예: 패스트푸드 신메뉴, 음료 당분 등)에 대한 응답 생성 금지.
- 출력: 반드시 아래 '요약 템플릿'의 <요약> 섹션만 출력합니다. (<질문>, <STT>는 출력 금지)
'''

SUMMARY_TEMPLATE = '''\
<요약>
1. **주 증상**
{symptoms_bullets}

2. **상담내용**
{counselling_bullets}

3. **심리상태**
{psych_bullets}

4. **AI 해석**
{ai_interp_bullets}

5. **주의사항**
{caution_bullets}
'''

SUMMARISE_PROMPT = ChatPromptTemplate.from_messages(
    [
      ("system", SYSTEM_PERSONA + "\n\n"
        "아래 '참고 문서(요약)'는 최신 지식 보강을 위한 참고용입니다. "
        "근거가 불명확하면 과도한 단정 대신 조심스러운 해석을 제시하세요.\n"
        "반드시 '요약 템플릿' 구조를 그대로 유지하며 <요약> 섹션만 출력하십시오."),
      ("human",
       "가이드 질문(고정 4개 중 선택): {guide_question}\n\n"
       "참고 문서(요약):\n{rag_context}\n\n"
       "사용자 STT 원문:\n{transcript}\n\n"
       "요구사항:\n"
       "1) '주 증상'에는 핵심 증상만 '-' 불릿으로 간결히.\n"
       "2) '상담내용'에는 구체적 사건/진술 중심으로 불릿화.\n"
       "3) '심리상태'는 **아래 제공된 불릿을 '그대로' 사용**합니다(문구/순서 임의 변경 금지). 비어있으면 스스로 작성.\n"
       "   제공 불릿:\n{psych_bullets_fixed}\n"
       "4) 'AI 해석'은 병명 단정 금지(예: '~가능성 시사').\n"
       "5) '주의사항'은 일상 안전, 정서적 지지, 검진 권고 등을 포함.\n"
       "6) 반드시 아래 형식으로 출력(오직 <요약> 섹션만):\n"
       "{summary_template}\n"
       "출력 시 한국어 따옴표(“)를 유지하세요."
      )
    ]
)
print("✅ 페르소나/템플릿 업데이트(섹션3 고정) - 완료")


✅ 페르소나/템플릿 업데이트(섹션3 고정) - 완료


In [244]:
# part 2 (2) 
#==========================================================
# 📌 LLM/임베딩/웹검색(RAG) 구성 + Multi-Search + Rerank-Then-Summarise
#     - DuckDuckGo는 ddgs 직접 호출로 경고 제거
#==========================================================
import os, sys, subprocess, json
import numpy as np
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

llm = ChatOpenAI(
    model=CHAT_MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_OUTPUT_TOKENS,
    api_key=OPENAI_API_KEY,
)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL_NAME, api_key=OPENAI_API_KEY)

# --- 공용 유틸 ---
def _try_import(module_path: str, cls_name: str):
    try:
        mod = __import__(module_path, fromlist=[cls_name])
        return getattr(mod, cls_name)
    except Exception:
        return None

def _norm_list_result(engine: str, item: Dict[str, Any]) -> Document:
    title   = item.get("title") or item.get("name") or ""
    link    = item.get("link") or item.get("url") or item.get("href") or item.get("source") or ""
    snippet = item.get("snippet") or item.get("body") or item.get("content") or item.get("description") or ""
    text = f"{title}\n{snippet}\nURL: {link}"
    return Document(page_content=text, metadata={"source": link, "title": title, "engine": engine})

def _result_to_docs(engine_name: str, result: Any) -> List[Document]:
    docs: List[Document] = []
    if isinstance(result, list):
        for it in result:
            if isinstance(it, dict):
                docs.append(_norm_list_result(engine_name, it))
            else:
                docs.append(Document(page_content=str(it), metadata={"engine": engine_name}))
    else:
        docs.append(Document(page_content=str(result), metadata={"engine": engine_name}))
    return docs

# --- ddgs (DuckDuckGo native) ---
def _ensure_ddgs():
    try:
        from ddgs import DDGS
        return DDGS
    except Exception:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "ddgs"], stdout=subprocess.DEVNULL)
            from ddgs import DDGS
            return DDGS
        except Exception:
            return None

DDGS = _ensure_ddgs()

def ddgs_search(query: str, k: int = 5) -> List[Document]:
    if DDGS is None:
        return []
    docs: List[Document] = []
    try:
        with DDGS() as ddgs:
            # 참고: ddgs.text(...)는 dict 스트림을 반환 (title, href, body 등)
            for i, item in enumerate(ddgs.text(query, max_results=k)):
                title = item.get("title") or ""
                link  = item.get("href") or ""
                body  = item.get("body") or ""
                docs.append(Document(page_content=f"{title}\n{body}\nURL: {link}",
                                     metadata={"source": link, "title": title, "engine": "ddgs"}))
                if i+1 >= k:
                    break
    except Exception as e:
        print(f"⚠️ ddgs 검색 실패: {e}")
    return docs

# --- Google CSE / SerpAPI / Bing (있을 때만 사용) ---
GoogleSearchAPIWrapper = _try_import("langchain_community.utilities", "GoogleSearchAPIWrapper")
SerpAPIWrapper         = _try_import("langchain_community.utilities", "SerpAPIWrapper")
BingSearchAPIWrapper   = _try_import("langchain_community.utilities", "BingSearchAPIWrapper")
WikipediaLoader        = _try_import("langchain_community.document_loaders", "WikipediaLoader")
Chroma                 = _try_import("langchain_community.vectorstores", "Chroma")

def _safe_init_google():
    if GoogleSearchAPIWrapper and os.getenv("GOOGLE_API_KEY") and os.getenv("GOOGLE_CSE_ID"):
        try:
            return GoogleSearchAPIWrapper()
        except Exception:
            return None
    return None

def _safe_init_serpapi():
    if SerpAPIWrapper and os.getenv("SERPAPI_API_KEY"):
        try:
            return SerpAPIWrapper()
        except Exception:
            return None
    return None

def _safe_init_bing():
    if BingSearchAPIWrapper and os.getenv("BING_SUBSCRIPTION_KEY"):
        try:
            return BingSearchAPIWrapper()
        except Exception:
            return None
    return None

gse  = _safe_init_google()
serp = _safe_init_serpapi()
bing = _safe_init_bing()

def multi_engine_search(query: str, k: int = 5) -> List[Document]:
    docs: List[Document] = []
    # 우선순위: Google CSE → Bing → SerpAPI → ddgs
    if gse:
        try:
            res = gse.results(query, k)
            docs.extend(_result_to_docs("google", res))
        except Exception as e:
            print(f"⚠️ google 검색 실패: {e}")
    if bing:
        try:
            res = bing.results(query, k)
            docs.extend(_result_to_docs("bing", res))
        except Exception as e:
            print(f"⚠️ bing 검색 실패: {e}")
    if serp:
        try:
            res = serp.results(query, k)
            docs.extend(_result_to_docs("serpapi", res))
        except Exception as e:
            print(f"⚠️ serpapi 검색 실패: {e}")
    # ddgs는 항상 마지막 폴백으로 실행
    docs.extend(ddgs_search(query, k))

    # 간단 중복 제거
    seen = set()
    uniq_docs: List[Document] = []
    for d in docs:
        key = (d.metadata.get("source") or "") + "|" + (d.metadata.get("title") or "") + "|" + d.page_content[:80]
        if key in seen:
            continue
        seen.add(key)
        uniq_docs.append(d)
    return uniq_docs

def _embed_texts(texts: List[str]) -> List[List[float]]:
    return embeddings.embed_documents(texts)

def _embed_query(text: str) -> List[float]:
    return embeddings.embed_query(text)

def _cosine(a: List[float], b: List[float]) -> float:
    a = np.array(a, dtype=np.float32); b = np.array(b, dtype=np.float32)
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + 1e-12
    return float(np.dot(a, b) / denom)

def build_rag_context_rerank(
    transcript: str,
    queries: List[str],
    k_search: int = 8,
    k_vec: int    = 12,
    k_rerank: int = 8,
    k_final: int  = 4
) -> str:
    # 1) 멀티 엔진 검색 집계
    all_docs: List[Document] = []
    for q in queries:
        all_docs.extend(multi_engine_search(q, k=max(3, k_search // max(1, len(queries)) + 1)))

    # 2) 위키 보강
    if (WikipediaLoader is not None) and (len(all_docs) < 2):
        try:
            wiki_docs = WikipediaLoader(query="치매", load_max_docs=3).load()
            all_docs.extend(wiki_docs)
        except Exception as e:
            print(f"⚠️ Wikipedia 로드 실패: {e}")

    if not all_docs:
        return ""

    # 3) 1차 후보 (Chroma 사용 가능 시)
    if Chroma is not None:
        try:
            vs = Chroma.from_documents(all_docs, embeddings)
            retriever = vs.as_retriever(search_kwargs={"k": min(k_vec, max(1, len(all_docs)))})
            seed_query = " ".join(queries[:3]) or "치매 경도인지장애 초기 증상 안전 가족 교육 평가도구"
            vec_docs = retriever.invoke(seed_query)
        except Exception as e:
            print(f"⚠️ 벡터스토어/리트리버 생성 실패: {e}")
            vec_docs = all_docs[:k_vec]
    else:
        vec_docs = all_docs[:k_vec]

    # 4) 재랭크 (쿼리+원문 기반 코사인)
    combined_query = ((" ".join(queries)) + " " + transcript[:600]).strip()
    q_emb = _embed_query(combined_query)
    cand_docs  = vec_docs[:k_rerank]
    cand_texts = [d.page_content for d in cand_docs]
    cand_embs  = _embed_texts(cand_texts)
    scores = [(_cosine(q_emb, e), i) for i, e in enumerate(cand_embs)]
    scores.sort(key=lambda x: x[0], reverse=True)

    top_docs = [cand_docs[i] for (s, i) in scores[:k_final]]
    rag_text = "\n\n".join([d.page_content[:1200] for d in top_docs])
    return rag_text

print("✅ LLM/임베딩/멀티검색(+ddgs, 재랭크) - 준비 완료")


✅ LLM/임베딩/멀티검색(+ddgs, 재랭크) - 준비 완료


In [ ]:
# part 2 (3)
#==========================================================
# 📌 모델 기반 온토픽 감지기 (무하드코딩)
#   - Zero-shot LLM 분류(점수+근거스팬)
#   - 세션 prior(가이드 질문 이후 가산)만 사용
#==========================================================
import json, time
from dataclasses import dataclass
from typing import Optional, Dict, List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

CLASSIFY_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "너는 입력이 ‘치매/인지장애 상담’인지 판단하는 분류기다. "
     "판단 기준: (a) 기억력 저하/단어 회상 곤란/언어 유창성 저하/방향감각 문제/일상생활 곤란/정서반응 등의 구체적 서술, "
     "(b) 보호자 관찰/검사/평가/안전 문제/가족 교육. "
     "메타 대화(너는 누구냐, 상담 되냐 등), 일반 잡담, 비치매 일반 건강/일상은 off_topic. "
     "오직 JSON만 출력: "
     "{\"on_topic\": true|false, \"score\": 0.0~1.0, "
     "\"evidence_spans\": [\"...\", \"...\"], \"reason\": \"...\"}. "
     "evidence_spans는 원문 발췌 1~3개(각 6~40자)."),
    ("human", "입력:\n\"\"\"\n{user_text}\n\"\"\"\n오직 JSON:")
])

judge_llm = ChatOpenAI(model=CHAT_MODEL_NAME, temperature=0.0, max_tokens=220, api_key=OPENAI_API_KEY)
judge_chain = CLASSIFY_PROMPT | judge_llm | StrOutputParser()

@dataclass
class DetectResult:
    label: str
    on_topic_prob: float
    stage_scores: Dict[str,float]
    evidence: List[str]

DEFAULT_PRIOR = 0.60
SESSION_STATE: Dict[str, Dict] = {}

def get_session(session_id: Optional[str]) -> Dict:
    sid = session_id or "default"
    if sid not in SESSION_STATE:
        SESSION_STATE[sid] = {"prior": DEFAULT_PRIOR, "last_ts": time.time(), "history": []}
    return SESSION_STATE[sid]

def mark_guide_question_shown(session_id: Optional[str]):
    s = get_session(session_id)
    s["prior"] = max(s["prior"], 0.75)  # 가이드 직후 응답은 on-topic prior↑

def update_session(session_id: Optional[str], label: str, prob: float):
    s = get_session(session_id)
    s["history"].append({"label": label, "prob": prob})
    s["last_ts"] = time.time()
    last = s["history"][-3:]
    s["prior"] = 0.5 * s["prior"] + 0.5 * (sum(h["prob"] for h in last) / len(last))

TAU_ON = 0.60           # 최종 on-topic 임계
BAND   = (0.48, 0.60)   # 회색지대 → 안내(Abstain)

def detect_topic(text: str, session_id: Optional[str]=None) -> DetectResult:
    t = (text or "").strip()
    if not t:
        return DetectResult("off_topic", 0.0, {"llm": 0.0}, [])

    prior = get_session(session_id)["prior"]
    try:
        raw = judge_chain.invoke({"user_text": t})
        data = json.loads(raw)
        score = float(data.get("score", 0.5))
        evid  = [e for e in (data.get("evidence_spans") or []) if isinstance(e, str)]
        combined = max(0.0, min(1.0, 0.65 * score + 0.35 * prior))
    except Exception:
        # LLM 실패 시 prior만으로 판정(가이드 직후 누락 방지)
        evid = []
        combined = prior

    if combined >= TAU_ON:
        label = "on_topic"
    elif BAND[0] <= combined < BAND[1]:
        label = "abstain"
    else:
        label = "off_topic"

    return DetectResult(label, combined, {"llm": combined}, evid)

print("✅ 온토픽 감지기(모델 기반) - 준비 완료")


✅ 감지기(완화 튜닝) - 준비 완료


In [246]:
# part 3 (1) 
#==========================================================
# 📌 검색질의 생성기 (STT → 관련 검색어 2~4개)
#==========================================================
QUERY_PROMPT = ChatPromptTemplate.from_messages(
    [
      ("system",
       "너는 의료 상담 보조 검색어 생성기이다. 입력 STT에서 핵심 증상/키워드를 뽑아 "
       "치매/경도인지장애와 관련된 한국어 검색질의 2~4개를 JSON 배열로만 출력하라."),
      ("human", "{transcript}")
    ]
)
query_llm   = ChatOpenAI(model=CHAT_MODEL_NAME, temperature=0.1, max_tokens=120, api_key=os.getenv("OPENAI_API_KEY"))
query_chain = QUERY_PROMPT | query_llm | StrOutputParser()

def make_search_queries(transcript: str) -> List[str]:
    try:
        raw = query_chain.invoke({"transcript": transcript})
        qs = json.loads(raw)
        qs = [q for q in qs if isinstance(q, str)]
        base = ["치매 초기 증상", "경도인지장애 언어 유창성", "일상 안전 가족 교육", "단기 기억력 저하 원인"]
        return (qs + base)[:4]
    except Exception:
        return ["치매 초기 증상", "경도인지장애 언어 유창성", "일상 안전 가족 교육", "단기 기억력 저하 원인"]

print("✅ 검색질의 생성기 - 준비 완료")


✅ 검색질의 생성기 - 준비 완료


In [ ]:
# part 3 (1.5)
#==========================================================
# 📌 심리상태 추출기 (감정-근거 스팬 JSON → 불릿 문자열)
#==========================================================
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

EMOTION_SET = ["두려움","불안","걱정","수치심","당황","답답","무기력","혼란","자책"]

PSYCH_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "다음 한국어 상담 발화에서 감정-근거 쌍을 1~4개 추출하라. "
     "감정(emotion)은 다음 중에서 고르라: " + ",".join(EMOTION_SET) + ". "
     "근거(evidence)는 **반드시 원문에서 6~30자 직발췌**. "
     "JSON 배열만 출력. 예: [{\"emotion\":\"두려움\",\"evidence\":\"혼자 외출하는 것도 무섭습니다\"}, ...] "
     "증거가 불충분하면 빈 배열[]을 출력하라."),
    ("human", "{transcript}")
])
_psych_llm = ChatOpenAI(model=CHAT_MODEL_NAME, temperature=0.1, max_tokens=200, api_key=OPENAI_API_KEY)
_psych_chain = PSYCH_PROMPT | _psych_llm | StrOutputParser()

def build_psych_bullets(transcript: str) -> str:
    # 1) LLM 추출
    pairs = []
    try:
        raw = _psych_chain.invoke({"transcript": transcript})
        data = json.loads(raw)
        if isinstance(data, list):
            for it in data:
                emo = (it.get("emotion") or "").strip()
                ev  = (it.get("evidence") or "").strip().strip('“"').strip()
                if emo and ev and 6 <= len(ev) <= 40:
                    pairs.append((emo, ev))
    except Exception:
        pairs = []

    # 2) 휴리스틱 보강(없을 때)
    if not pairs:
        heuristics = [
            ("두려움",  r"(무섭|두렵)"),
            ("불안",    r"불안"),
            ("걱정",    r"걱정"),
            ("당황",    r"당황"),
            ("수치심",  r"(창피|수치)"),
            ("답답",    r"답답"),
        ]
        sents = [s.strip() for s in re.split(r"[.!?\n]", transcript) if s.strip()]
        for emo, pat in heuristics:
            rex = re.compile(pat)
            for s in sents:
                if rex.search(s) and 6 <= len(s) <= 40:
                    pairs.append((emo, s))
                    break
            if len(pairs) >= 3:
                break

    # 3) 불릿 문자열 구성
    if not pairs:
        return ""  # 프롬프트가 비었으면 LLM이 생성

    lines = []
    for emo, ev in pairs[:4]:
        lines.append(f"- {emo}\n  - 근거 : “{ev}”")
    return "\n".join(lines)


✅ 심리상태 추출기(한국어 LLM 도출형) - 준비 완료


In [250]:
# part 3 (2)
#==========================================================
# 📌 요약 체인 (섹션3 고정 불릿 전달, 변수 누락 방지)
#==========================================================
summary_chain = SUMMARISE_PROMPT | llm | StrOutputParser()

ABSTAIN_MESSAGE = (
    "치매 관련 증상·경험 중심으로 조금만 더 구체적으로 말씀해 주시면 요약을 도와드리겠습니다.\n"
    "예) 최근에 잊어버린 사례, 단어가 막혔던 상황, 외출 시 불편했던 점 등"
)

def summarise_transcript_only_summary(
    transcript: str,
    guide_question_index: int = 3,
    session_id: Optional[str] = None,
) -> str:
    # 1) 온토픽 감지
    detect = detect_topic(transcript, session_id=session_id)

    if detect.label == "off_topic":
        update_session(session_id, "off_topic", detect.on_topic_prob)
        return OFF_TOPIC_MESSAGE

    if detect.label == "abstain":
        update_session(session_id, "abstain", detect.on_topic_prob)
        return ABSTAIN_MESSAGE

    update_session(session_id, "on_topic", detect.on_topic_prob)

    # 2) RAG 컨텍스트
    guide_question = GUIDE_QUESTIONS[max(0, min(3, int(guide_question_index)))]
    queries = make_search_queries(transcript)
    rag_context = build_rag_context_rerank(transcript, queries)

    # 3) 섹션3(심리상태) 고정 불릿 생성 — 반드시 변수로 전달
    try:
        psych_bullets_fixed = build_psych_bullets(transcript).strip()
    except Exception:
        psych_bullets_fixed = ""
    if not psych_bullets_fixed:
        psych_bullets_fixed = "(없음)"

    # 4) 체인 호출 (모든 변수 전달)
    out = summary_chain.invoke({
        "transcript": transcript.strip(),
        "rag_context": (rag_context.strip() if rag_context else "(문맥 없음)"),
        "guide_question": guide_question,
        "summary_template": SUMMARY_TEMPLATE,
        "psych_bullets_fixed": psych_bullets_fixed,   # ✅ 필수 전달
    })
    return out.strip()


In [251]:
# part 4 (demo)
#==========================================================
# 📌 데모 실행: './script_1.txt' → 'summary_1_.txt' 로 저장
#==========================================================
INPUT_TXT  = "./script_1.txt"
OUTPUT_TXT = "./summary_1_.txt"
QUESTION_INDEX = 3
SESSION_ID = "demo-session-1"

# 가이드 질문을 던진 것으로 간주 → 세션 prior boost
mark_guide_question_shown(SESSION_ID)

if not os.path.exists(INPUT_TXT):
    print(f"⚠️ {INPUT_TXT} 파일이 없습니다. 테스트용으로 데모 텍스트를 생성합니다.")
    demo_stt = (
        "요즘 물건을 어디에 두었는지 자꾸 잊어버려서 힘듭니다. "
        "안경이나 휴대폰을 두고도 찾지를 못해서 가족들한테 자주 물어보게 돼요. "
        "또 밖에 나갔다가 길을 잃을까 봐 혼자 외출하는 것도 무섭습니다. "
        "집에서는 전화 통화를 하고 나면 금방 무슨 이야기를 했는지 기억이 안 나서 답답하고요. "
        "제 상태가 점점 더 나빠져서 가족들에게 짐이 될까 봐 가장 걱정입니다. "
        "대화할 때 단어가 잘 떠오르지 않아서 사람들이 이상하게 생각할까 봐 창피하기도 합니다."
    )
    with open(INPUT_TXT, "w", encoding="utf-8") as f:
        f.write(demo_stt)

out_path = summarise_from_txt_and_save(INPUT_TXT, OUTPUT_TXT,
                                       guide_question_index=QUESTION_INDEX, session_id=SESSION_ID)
print(f"✅ 저장 완료: {out_path}")
with open(out_path, 'r', encoding='utf-8') as f:
    preview = f.read()
print('------ 미리보기 ------')
print(preview[:1200])


✅ 저장 완료: ./summary_1_.txt
------ 미리보기 ------
<요약>
1. **주 증상**
- 물건을 잃어버림
- 외출 시 길을 잃을까 두려움
- 전화 통화 후 기억 상실
- 단어가 잘 떠오르지 않음

2. **상담내용**
- 안경이나 휴대폰을 두고 찾지 못해 가족에게 자주 물어봄.
- 혼자 외출하는 것이 무섭고 길을 잃을까 걱정됨.
- 전화 통화를 하고 나면 대화 내용을 금방 잊어버림.
- 자신의 상태가 나빠져 가족에게 짐이 될까 걱정함.
- 대화 중 단어가 잘 생각나지 않아 창피함.

3. **심리상태**
(없음)

4. **AI 해석**
현재의 증상은 경도인지장애(MCI)와 관련이 있을 수 있으며, 조기 진단과 치료가 필요할 수 있습니다.

5. **주의사항**
- 일상에서 물건을 두는 자리를 정해두고, 외출 시에는 안전한 경로를 미리 계획하는 것이 좋습니다.
- 가족과의 소통을 통해 정서적 지지를 받는 것이 중요합니다.
- 증상이 지속되거나 악화될 경우 전문의와 상담하여 검진을 받는 것을 권장합니다.



In [252]:
# part 5 (optional cli)
#==========================================================
# 📌 간단 대화 루프 (오프토픽 차단 + 요약)
#==========================================================
# 사용법:
#  - 첫 발화는 4개 고정 질문 중 하나를 출력
#  - 이후 사용자의 긴 응답(텍스트)을 받아 요약 템플릿으로 정리

def start_chat(guide_question_index: int = 3):
    q = GUIDE_QUESTIONS[max(0, min(3, int(guide_question_index)))]
    print(f"[상담 챗봇] {q}")
    print("[안내] 긴 답변을 입력하세요. '/quit' 입력 시 종료.")
    while True:
        user = input("\n[사용자] ").strip()
        if user.lower() in {"/quit", "quit", "exit"}:
            print("종료합니다.")
            break
        result = summarise_transcript(user, guide_question_index=guide_question_index)
        print("\n[요약]\n")
        print(result)

# start_chat(guide_question_index=3)  # ← 필요 시 주석 해제하여 실행
print("✅ (선택) CLI 루프 준비됨 — 필요 시 start_chat() 실행")


✅ (선택) CLI 루프 준비됨 — 필요 시 start_chat() 실행


In [256]:
# part 6
#==========================================================
# 📌 파일 입력 전용 실행 (오프토픽 차단 + 요약 저장)
#   - 반복문(대화 루프) 제거
#   - ./script_1.txt → 요약 → ./summary_1_.txt (또는 자동 파일명)
#==========================================================
import os

# [설정값]
INPUT_TXT            = "./script_1.txt"     # ✅ 실제 STT 텍스트 파일 경로
# INPUT_TXT            = "./개소리4.txt"     # ✅ 실제 STT 텍스트 파일 경로
# INPUT_TXT            = "./르세라핌_smart.txt"     # ✅ 실제 STT 텍스트 파일 경로
OUTPUT_DIR           = "."                  # 저장 폴더
USE_AUTO_NAMING      = False                # True: summary_<원본파일명>.txt, False: 고정 파일명 사용
OUTPUT_FIXED_NAME    = "summary_1.txt"     # USE_AUTO_NAMING=False일 때 사용
GUIDE_QUESTION_INDEX = 1                    # 0~3 (Q1~Q4)
SESSION_ID           = "file-run-session-1" # 세션 ID
SKIP_OFFTOPIC_SAVE   = True                 # 오프토픽이면 저장하지 않음

# 1) 가이드 질문을 던진 것으로 간주 → 세션 prior boost
mark_guide_question_shown(SESSION_ID)

# 2) 입력 파일 확인
if not os.path.exists(INPUT_TXT):
    raise FileNotFoundError(f"입력 파일이 존재하지 않습니다: {INPUT_TXT}")

# 3) 요약 생성 (오프토픽/보류 안내 포함)
summary_only = summarise_from_txt(
    INPUT_TXT,
    guide_question_index=GUIDE_QUESTION_INDEX,
    session_id=SESSION_ID,
)

# 4) 오프토픽 처리(저장 여부)
if summary_only.strip() == OFF_TOPIC_MESSAGE and SKIP_OFFTOPIC_SAVE:
    print("⚠️ 오프토픽으로 판정되어 저장을 생략합니다.")
    print("메시지:", OFF_TOPIC_MESSAGE)
else:
    # 5) 출력 경로 결정
    if USE_AUTO_NAMING:
        base = os.path.splitext(os.path.basename(INPUT_TXT))[0]
        out_name = f"summary_{base}.txt"
    else:
        out_name = OUTPUT_FIXED_NAME

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_path = os.path.join(OUTPUT_DIR, out_name)

    # 6) 저장
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(summary_only.strip() + "\n")

    print(f"✅ 요약 저장 완료: {out_path}\n")
    # 7) 미리보기    preview = summary_only.strip()
    print("------ 요약 미리보기 ------")
    print(preview[:1200])


✅ 요약 저장 완료: .\summary_1.txt

------ 요약 미리보기 ------
<요약>
1. **주 증상**
- 물건을 잃어버림
- 외출 시 길을 잃을까 두려움
- 전화 통화 후 기억 상실
- 단어가 잘 떠오르지 않음

2. **상담내용**
- 안경이나 휴대폰을 두고 찾지 못해 가족에게 자주 물어봄.
- 혼자 외출하는 것이 무섭고 길을 잃을까 걱정됨.
- 전화 통화를 하고 나면 대화 내용을 금방 잊어버림.
- 자신의 상태가 나빠져 가족에게 짐이 될까 걱정함.
- 대화 중 단어가 잘 생각나지 않아 창피함.

3. **심리상태**
(없음)

4. **AI 해석**
현재의 증상은 경도인지장애(MCI)와 관련이 있을 수 있으며, 조기 진단과 치료가 필요할 수 있습니다.

5. **주의사항**
- 일상에서 물건을 두는 자리를 정해두고, 외출 시에는 안전한 경로를 미리 계획하는 것이 좋습니다.
- 가족과의 소통을 통해 정서적 지지를 받는 것이 중요합니다.
- 증상이 지속되거나 악화될 경우 전문의와 상담하여 검진을 받는 것을 권장합니다.

